In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing necessary Libraries

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import cv2
from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# storing dateset Images and Labels in lists

In [3]:
dataset_dir = '/kaggle/input/5-flower-types-classification-dataset/flower_images'#string containing the path of our main directory
image_size = 224
batch = 64
x = []
y = []
class_names = sorted(os.listdir(dataset_dir)) #gives the sorted list of sub directories in the directory
for class_name in class_names :
    class_dir = os.path.join(dataset_dir,class_name) #gives path of each sub-directory in the dataset_dir directory
    images = os.listdir(class_dir) #gives a list of contents of the sub directories i.e the list of images in this case
    for image_name in images:
        image_path = os.path.join(class_dir,image_name) #gives path of the image
        image = cv2.imread(image_path) #gives us the image from path in matrix form
        image = cv2.resize(image,(image_size, image_size)) #ensures constant size of all the images
        image  = image / 255.0 #rescaling images to reduce number of parameters to make it easier for the model to train
        x.append(image) # storing the images in the list x
        y.append(class_name) #storing the class_name of the corresponding imagein list x

# Preprocessing and One Hot Encoding (to avoid ordinality in classes)

In [4]:
from sklearn.preprocessing import LabelEncoder #for transforming classes of object type to intiger
from tensorflow.keras.utils import to_categorical #for performing one hot encoding on class labels

label_encoding = LabelEncoder()
y_encoded = label_encoding.fit_transform(y) #forming a list containing labels in intiger forrmat

x = np.array(x)
y_encoded = np.array(y_encoded)

print(label_encoding.classes_)
num_classes = len(label_encoding.classes_)

y_encoded = to_categorical(y_encoded, num_classes = num_classes)

x_train, x_test, y_train, y_test = train_test_split(x , y_encoded, test_size = 0.2, random_state = 42) #splaiitng training and resting data

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

['Lilly' 'Lotus' 'Orchid' 'Sunflower' 'Tulip']
x_train shape: (4000, 224, 224, 3)
x_test shape: (1000, 224, 224, 3)
y_train shape: (4000, 5)
y_test shape: (1000, 5)


# Training the NN

In [5]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (224, 224, 3)))
model.add(MaxPooling2D(pool_size = (3, 3)))
model.add(Conv2D(filters = 100, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 100, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Flatten())
model.add(Dense(num_classes, activation = 'softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 74, 74, 100)       57700     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 100)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 100)       90100     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 100)        0

In [6]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy')
model.fit(x_train, y_train, batch_size = 10, epochs = 5, validation_data = (x_test, y_test))

Epoch 1/5
400/400 [==============================] - 234s 581ms/step - loss: 1.2517 - accuracy: 0.4780 - val_loss: 1.0565 - val_accuracy: 0.5650
Epoch 2/5
400/400 [==============================] - 230s 576ms/step - loss: 0.9901 - accuracy: 0.6127 - val_loss: 1.0007 - val_accuracy: 0.6040
Epoch 3/5
400/400 [==============================] - 228s 570ms/step - loss: 0.8201 - accuracy: 0.6945 - val_loss: 0.8063 - val_accuracy: 0.6920
Epoch 4/5
400/400 [==============================] - 228s 571ms/step - loss: 0.6100 - accuracy: 0.7740 - val_loss: 0.6428 - val_accuracy: 0.7710
Epoch 5/5
400/400 [==============================] - 230s 575ms/step - loss: 0.4368 - accuracy: 0.8430 - val_loss: 0.6804 - val_accuracy: 0.7780


In [7]:
test_loss, test_accuracy = model.evaluate(x_test,y_test)
print(test_loss)
print(test_accuracy)

32/32 [==============================] - 16s 497ms/step - loss: 0.6804 - accuracy: 0.7780
0.6803770065307617
0.777999997138977
